In [ ]:
# 피클과 동영상 갯수 맞춰보기
from glob import glob
from collections import Counter
from IPython.display import HTML
from pathlib import Path

def display_video(p): 
    display(HTML(f"""<video src="{p}" width=640 controls autoplay/>"""))

#mp4s = sorted(glob('split_side/C0023*.mp4'))
mp4s = sorted(glob('khd/clip/**/*.mp4', recursive=True))
df_face   = glob(f'khd/df_info/df_face_info/*.pickle')
df_anchor = glob(f'khd/df_info/df_anchor_i/*.pickle')

print(f'df_face:{len(set(df_face))}, df_anchor:{len(set(df_anchor))}, mp4s:{len(set(mp4s))}')

In [ ]:
# 두개 이상으로 분리된 영상 보기 
df_anchor_names = [Path(e).stem.split('_')[0] for e in df_anchor]
print(len(df_anchor_names))

for k, v in Counter(df_anchor_names).items():
    if v == 1:
        continue
    for d in [e for e in mp4s if Path(e).stem == k]:
        print(d)
        display_video(d)

In [ ]:
# 결과 샘플 보기
from pathlib import Path
import imageio, cv2, random, pandas as pd
from tqdm.auto import tqdm
from moviepy.editor import ImageSequenceClip
import os

%pdb on
sample_pickle = random.choice(df_anchor)
#sample_pickle = 'df_anchor_i/Est_Soft_18_2.mov.13-23.ss-06-28.to-07-02_000.pickle'
sample_mp4 = [f for f in mp4s if Path(f).stem == Path(sample_pickle).stem[:-4]][0]
print(sample_pickle)
print(sample_mp4)
frames = imageio.mimread(sample_mp4, memtest=False)
for p in [sample_pickle]:
    df = pd.read_pickle(p)
    print(p)
    for _, row in tqdm(df.iterrows(), total=len(df)):
        im = frames[row['frame_idx']]
        x1, y1, x2, y2 = row['box']
        cv2.rectangle(im, (x1, y1), (x2, y2), (255,0,0))
    clip = ImageSequenceClip(frames, fps=30)
    out_name = f'temp/{Path(p).name}.mp4'
    os.makedirs(os.path.dirname(out_name), exist_ok=True)
    clip.write_videofile(out_name)#, logger=None)
    display_video(out_name)